In [1]:
import os, time
import pandas
import tensorflow as tf
import tensorflow_hub as hub
from kaggle_datasets import KaggleDatasets
print(tf.version.VERSION)

2.1.0


Detect TPUs or GPUs:

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


Set maximum sequence length and path variables.

In [3]:
SEQUENCE_LENGTH = 128

# Note that private datasets cannot be copied - you'll have to share any pretrained models 
# you want to use with other competitors!
GCS_PATH = KaggleDatasets().get_gcs_path('jigsaw-multilingual-toxic-comment-classification')
BERT_GCS_PATH = KaggleDatasets().get_gcs_path('bert-multi')
BERT_GCS_PATH_SAVEDMODEL = BERT_GCS_PATH + "/bert_multi_from_tfhub"

Define the model. We convert m-BERT's output to a final probabilty estimate. We're using an [m-BERT model from TensorFlow Hub](https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/1).

In [4]:
def multilingual_bert_model(max_seq_length=SEQUENCE_LENGTH, trainable_bert=True):
    """Build and return a multilingual BERT model and tokenizer."""
    input_word_ids = tf.keras.layers.Input(
        shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.layers.Input(
        shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.layers.Input(
        shape=(max_seq_length,), dtype=tf.int32, name="all_segment_id")
    
    # Load a SavedModel on TPU from GCS. This model is available online at 
    # https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/1. You can use your own 
    # pretrained models, but will need to add them as a Kaggle dataset.
    bert_layer = tf.saved_model.load(BERT_GCS_PATH_SAVEDMODEL)
    # Cast the loaded model to a TFHub KerasLayer.
    bert_layer = hub.KerasLayer(bert_layer, trainable=trainable_bert)

    pooled_output, outputs = bert_layer([input_word_ids, input_mask, segment_ids])
    print("outputs.shape", outputs.shape)
    print("pooled_output.shape", pooled_output.shape)
    
    #outputs = tf.keras.layers.GRU(128) (outputs)
    #outputs = tf.keras.layers.Dropout(0.5) (outputs)
    #print("outputs.shape after lstms", outputs.shape)
    #outputs = tf.keras.layers.Flatten() (outputs)
    outputs = tf.keras.layers.GlobalMaxPooling1D() (outputs)
    print("outputs.shape after flatten", outputs.shape)
    outputs = tf.keras.layers.Dense(32, activation='relu')(outputs)
    
    pooled_output = tf.keras.layers.Dense(32, activation='relu')(pooled_output)
    
    output = tf.keras.layers.Concatenate() ([pooled_output, outputs])
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='labels')(output)

    return tf.keras.Model(inputs={'input_word_ids': input_word_ids,
                                  'input_mask': input_mask,
                                  'all_segment_id': segment_ids},
                          outputs=output)

Load the preprocessed dataset. See the demo notebook for sample code for performing this preprocessing.

In [5]:
def parse_string_list_into_ints(strlist):
    s = tf.strings.strip(strlist)
    s = tf.strings.substr(
        strlist, 1, tf.strings.length(s) - 2)  # Remove parentheses around list
    s = tf.strings.split(s, ',', maxsplit=SEQUENCE_LENGTH)
    s = tf.strings.to_number(s, tf.int32)
    s = tf.reshape(s, [SEQUENCE_LENGTH])  # Force shape here needed for XLA compilation (TPU)
    return s

def format_sentences(data, label='toxic', remove_language=False):
    labels = {'labels': data.pop(label)}
    if remove_language:
        languages = {'language': data.pop('lang')}
    # The remaining three items in the dict parsed from the CSV are lists of integers
    for k,v in data.items():  # "input_word_ids", "input_mask", "all_segment_id"
        data[k] = parse_string_list_into_ints(v)
    return data, labels

def make_sentence_dataset_from_csv(filename, label='toxic', language_to_filter=None):
    # This assumes the column order label, input_word_ids, input_mask, segment_ids
    SELECTED_COLUMNS = [label, "input_word_ids", "input_mask", "all_segment_id"]
    label_default = tf.int32 if label == 'id' else tf.float32
    COLUMN_DEFAULTS  = [label_default, tf.string, tf.string, tf.string]

    if language_to_filter:
        insert_pos = 0 if label != 'id' else 1
        SELECTED_COLUMNS.insert(insert_pos, 'lang')
        COLUMN_DEFAULTS.insert(insert_pos, tf.string)

    preprocessed_sentences_dataset = tf.data.experimental.make_csv_dataset(
        filename, column_defaults=COLUMN_DEFAULTS, select_columns=SELECTED_COLUMNS,
        batch_size=1, num_epochs=1, shuffle=False)  # We'll do repeating and shuffling ourselves
    # make_csv_dataset required a batch size, but we want to batch later
    preprocessed_sentences_dataset = preprocessed_sentences_dataset.unbatch()
    
    if language_to_filter:
        preprocessed_sentences_dataset = preprocessed_sentences_dataset.filter(
            lambda data: tf.math.equal(data['lang'], tf.constant(language_to_filter)))
        #preprocessed_sentences.pop('lang')
    preprocessed_sentences_dataset = preprocessed_sentences_dataset.map(
        lambda data: format_sentences(data, label=label,
                                      remove_language=language_to_filter))

    return preprocessed_sentences_dataset

Set up our data pipelines for training and evaluation.

In [6]:
def make_dataset_pipeline(dataset, repeat_and_shuffle=True):
    """Set up the pipeline for the given dataset.
    
    Caches, repeats, shuffles, and sets the pipeline up to prefetch batches."""
    cached_dataset = dataset.cache()
    if repeat_and_shuffle:
        cached_dataset = cached_dataset.repeat().shuffle(2048)
    cached_dataset = cached_dataset.batch(32 * strategy.num_replicas_in_sync)
    cached_dataset = cached_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return cached_dataset

# Load the preprocessed English dataframe.
preprocessed_en_filename = (
    GCS_PATH + "/jigsaw-toxic-comment-train-processed-seqlen{}.csv".format(
        SEQUENCE_LENGTH))

# Set up the dataset and pipeline.
english_train_dataset = make_dataset_pipeline(
    make_sentence_dataset_from_csv(preprocessed_en_filename))

# Process the new datasets by language.
preprocessed_val_filename = (
    GCS_PATH + "/validation-processed-seqlen{}.csv".format(SEQUENCE_LENGTH))

nonenglish_val_datasets = {}
for language_name, language_label in [('Spanish', 'es'), ('Italian', 'it'),
                                      ('Turkish', 'tr')]:
    nonenglish_val_datasets[language_name] = make_sentence_dataset_from_csv(
        preprocessed_val_filename, language_to_filter=language_label)
    nonenglish_val_datasets[language_name] = make_dataset_pipeline(
        nonenglish_val_datasets[language_name])

nonenglish_val_datasets['Combined'] = tf.data.experimental.sample_from_datasets(
        (nonenglish_val_datasets['Spanish'], nonenglish_val_datasets['Italian'],
         nonenglish_val_datasets['Turkish']))

Compile our model. We'll first evaluate it on our new toxicity dataset in the 
different languages to see its performance. After that, we'll train it on one of our English datasets, and then again evaluate its performance on the new multilingual toxicity data. As our metric, we'll use the [AUC](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/AUC).

In [7]:
with strategy.scope():
    multilingual_bert = multilingual_bert_model()

    # Compile the model. Optimize using stochastic gradient descent.
    multilingual_bert.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(lr=1e-5),
        metrics=[tf.keras.metrics.AUC()])

multilingual_bert.summary()

outputs.shape (None, None, 768)
pooled_output.shape (None, 768)
outputs.shape after flatten (None, 768)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
all_segment_id (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 177853441   input_word_ids[0][0]    

In [8]:
# Test the model's performance on non-English comments before training.
#for language in nonenglish_val_datasets:
#    results = multilingual_bert.evaluate(nonenglish_val_datasets[language],
 #                                        steps=100, verbose=0)
  #  print('{} loss, AUC before training:'.format(language), results)

#results = multilingual_bert.evaluate(english_train_dataset,
    #                                 steps=100, verbose=0)
#print('\nEnglish loss, AUC before training:', results)

print()
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_auc', mode='max', verbose=1, patience=1)

# Train on English Wikipedia comment data.
history = multilingual_bert.fit(
    # Set steps such that the number of examples per epoch is fixed.
    # This makes training on different accelerators more comparable.
    english_train_dataset, steps_per_epoch=4000/strategy.num_replicas_in_sync, #4000
    epochs=10, verbose=1, validation_data=nonenglish_val_datasets['Combined'],
    validation_steps=100, callbacks=[es])   #100

# Re-evaluate the model's performance on non-English comments after training.
#for language in nonenglish_val_datasets:
 #   results = multilingual_bert.evaluate(nonenglish_val_datasets[language],
 #                                        steps=100, verbose=0)
  #  print('{} loss, AUC after training:'.format(language), results)

#results = multilingual_bert.evaluate(english_train_dataset,
  #                                   steps=100, verbose=0)
#print('\nEnglish loss, AUC after training:', results)

# trian on validation data too
es = EarlyStopping(monitor='auc', mode='max', verbose=1, patience=1)

# Train on English Wikipedia comment data.
history2 = multilingual_bert.fit(
    nonenglish_val_datasets['Combined'], steps_per_epoch=4000/strategy.num_replicas_in_sync, #4000
    epochs=10, verbose=1, callbacks=[es])   #100


Train for 500.0 steps, validate for 100 steps
Epoch 1/10


Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


500/500 [==============================] - 147s 294ms/step - loss: 0.1298 - auc: 0.9608 - val_loss: 0.4459 - val_auc: 0.8537
Epoch 2/10
500/500 [==============================] - 70s 141ms/step - loss: 0.1084 - auc: 0.9745 - val_loss: 0.4504 - val_auc: 0.8595
Epoch 3/10
500/500 [==============================] - 70s 140ms/step - loss: 0.0804 - auc: 0.9847 - val_loss: 0.5898 - val_auc: 0.8077
Epoch 00003: early stopping
Train for 500.0 steps
Epoch 1/10
500/500 [==============================] - 98s 196ms/step - loss: 0.0942 - auc: 0.9882
Epoch 2/10
500/500 [==============================] - 66s 132ms/step - loss: 0.0055 - auc: 0.9997
Epoch 3/10
500/500 [==============================] - 66s 132ms/step - loss: 0.0026 - auc: 0.9998
Epoch 4/10
500/500 [==============================] - 65s 131ms/step - loss: 0.0018 - auc: 0.9999
Epoch 5/10
500/500 [==============================] - 66s 132ms/step - loss: 0.0019 - auc: 0.9999
Epoch 00005: early stopping


# Generate predictions

Finally, we'll use our trained multilingual model to generate predictions for the test data.

In [9]:
import numpy as np

TEST_DATASET_SIZE = 63812

print('Making dataset...')
preprocessed_test_filename = (
    GCS_PATH + "/test-processed-seqlen{}.csv".format(SEQUENCE_LENGTH))
test_dataset = make_sentence_dataset_from_csv(preprocessed_test_filename, label='id')
test_dataset = make_dataset_pipeline(test_dataset, repeat_and_shuffle=False)

print('Computing predictions...')
test_sentences_dataset = test_dataset.map(lambda sentence, idnum: sentence)
probabilities = np.squeeze(multilingual_bert.predict(test_sentences_dataset))
print(probabilities)

print('Generating submission file...')
test_ids_dataset = test_dataset.map(lambda sentence, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_dataset.batch(TEST_DATASET_SIZE)))[
    'labels'].numpy().astype('U')  # All in one batch

np.savetxt('submission.csv', np.rec.fromarrays([test_ids, probabilities]),
           fmt=['%s', '%f'], delimiter=',', header='id,toxic', comments='')
!head submission.csv

Making dataset...
Computing predictions...
[3.3676624e-06 4.4703484e-07 9.9824965e-01 ... 2.7447939e-05 5.6624413e-07
 5.0663948e-07]
Generating submission file...
id,toxic
0,0.000003
1,0.000000
2,0.998250
3,0.000001
4,0.000000
5,0.000000
6,0.000001
7,0.000001
8,0.000003
